<a href="https://colab.research.google.com/github/Lauel09/IMDb-genre-classification/blob/main/testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
# import nltk stopwords and WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import unicodedata
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multiclass import OneVsRestClassifier
# import train test split
from sklearn.model_selection import train_test_split


In [5]:
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
import kaggle

In [6]:
# Download the dataset
!kaggle datasets download -d simhyunsu/imdbextensivedataset

Dataset URL: https://www.kaggle.com/datasets/simhyunsu/imdbextensivedataset
License(s): unknown
 93% 25.0M/26.9M [00:01<00:00, 20.4MB/s]
100% 26.9M/26.9M [00:01<00:00, 14.3MB/s]


In [7]:
import os
for name in os.listdir('/content'):
  print(name)

import zipfile

with zipfile.ZipFile('/content/imdbextensivedataset.zip','r') as zip_ref:
  zip_ref.extractall('.')

.config
kaggle.json
imdbextensivedataset.zip
sample_data


In [8]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

print(get_available_gpus())

[]


In [9]:

data = pd.read_csv("IMDb movies.csv",low_memory=False)
data = data.rename(columns={'genre':'genres','imdb_title_id':'imdb_id','date_published':'date','description':'plot'})
# drop every column except 'genre', 'imdb_title_id', 'title', 'year', 'date', 'duration', 'language', 'plot'
data = data[['genres', 'imdb_id', 'title', 'year', 'date', 'duration', 'language', 'plot']]


In [10]:
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("wordnet")
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def normalize_text(text):
    """
        Usual Unicode normalization.
    """
    try :
        return unicodedata.normalize('NFD', text).encode('ascii','ignore').decode('utf-8')
    except Exception as e:
        print(e)
        print(f"Caused error:{text}")
        return


# Preprocess text by tokenizing and lemmatizing it
def preprocess_text(text):
    # Tokenize, remove stopwords, and lemmatize
    tokens = nltk.word_tokenize(text.lower())
    filtered_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words and token.isalnum()]
    return ' '.join(filtered_tokens)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [11]:
# Drop every row with plot as NaN
data.drop(data[data['plot'].isna()].index, inplace=True)

# drop every rown with NaN value in it
data.dropna(inplace=True,axis=0)

# split the genres into a list, if it's already a list then leave it
data['genres'] = data['genres'].apply(lambda x: x.split(',') if type(x) is not list else x)

# preprocess the plot using WordNet()
data['plot'] = data['plot'].apply(lambda y: preprocess_text(y))

In [12]:
data = data # Now we will try all dataset
data.shape[0]
godfather_text = "The aging patriarch of an organized crime dynasty transfers control of his clandestine empire to his reluctant son."

tfidf_vectorizer = TfidfVectorizer(max_df=0.5,ngram_range=(1,2))
def transform_tfidf(text):
    transformed_text = tfidf_vectorizer.fit_transform(text)
    return transformed_text


from sklearn.preprocessing import MultiLabelBinarizer
# Convert y_train to binary array
mlb = MultiLabelBinarizer()
transformed_genres = mlb.fit_transform(data['genres'])

X_train,X_test,y_train,y_test = train_test_split(data['plot'],transformed_genres,test_size=0.2,random_state=21)

# transform the text using tfidf
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [13]:
data

,genres,imdb_id,title,year,date,duration,language,plot
3,"[Drama, History]",tt0002101,Cleopatra,1912,1912-11-13,100,English,fabled queen egypt affair roman general marc a...
4,"[Adventure, Drama, Fantasy]",tt0002130,L'Inferno,1911,1911-03-06,68,Italian,loosely adapted dante divine comedy inspired i...
5,"[Biography, Drama]",tt0002199,"From the Manger to the Cross; or, Jesus of Naz...",1912,1913,60,English,account life jesus christ based book new testa...
6,"[Biography, Drama, Romance]",tt0002423,Madame DuBarry,1919,1919-11-26,85,German,story madame dubarry mistress louis xv france ...
7,"[Drama, History]",tt0002445,Quo Vadis?,1913,1913-03-01,120,Italian,epic italian film quo vadis influenced many la...
...,...,...,...,...,...,...,...,...
85848,[Drama],tt9905462,Pengalila,2019,2019-03-08,111,Malayalam,unusual bond sixty year old dalit worker azhag...
85849,"[Comedy, Drama]",tt9906644,Manoharam,2019,2019-09-27,122,Malayalam,manoharan poster artist struggling find respec...
85850,[Comedy],tt9908390,Le lion,2020,2020-01-29,95,French,psychiatric hospital patient pretend crazy cha...
85851,"[Comedy, Drama]",tt9911196,De Beentjes van Sint-Hildegard,2020,2020-02-13,103,"German, Dutch",veterinary surgeon belief wife pampers much or...


In [14]:
# calculate total number of unique classes in data['genres']
total_classes = len(mlb.classes_)
total_classes


46

In [15]:
# calculate total number of unique classes in data['genres']
total_classes = len(mlb.classes_)
total_classes
# 44 unique classes
y_train.shape[1]

46

In [16]:
import keras
# import layers
from keras.models import Model,Sequential
from keras.layers import Dense,Input
import tensorflow as tf
import numpy as np

In [17]:
def return_sparse(csr_matrix):
  coo = csr_matrix.tocoo()
  indices = np.mat([coo.row,coo.col]).transpose()
  reorder_sparse = tf.sparse.reorder(tf.SparseTensor(indices,coo.data,coo.shape))
  return reorder_sparse

reorder_sparse = return_sparse(X_train_tfidf)


In [18]:
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

# Assuming y_train is your multi-label target variable
mlb = MultiLabelBinarizer()
y_train_bin = mlb.fit_transform(data['genres'])

class_weights = {}

for i in range(y_train_bin.shape[1]):
    class_weights[mlb.classes_[i]] = 1 / (np.sum(y_train_bin[:, i]) / len(y_train))

# Create a dictionary mapping class names to indices
class_indices = {class_name: index for index, class_name in enumerate(mlb.classes_)}
class_indices

new_weights = dict()
for (k,v) in class_weights.items():
    indices = class_indices.get(k)
    new_weights[indices] = v

new_weights

{0: 89.45479082321188,
 1: 16.858087487283825,
 2: 2071.4375,
 3: 215.9153094462541,
 4: 13.450892857142858,
 5: 12.288839451242122,
 6: 3.123751178133836,
 7: 20.56655290102389,
 8: 20.65627921470863,
 9: 104.7172195892575,
 10: 30.931404573028466,
 11: 15.1753663003663,
 12: 41.97973400886637,
 13: 39.62104004781829,
 14: 14.50142200831328,
 15: 66286.0,
 16: 22095.333333333332,
 17: 5.104813246053138,
 18: 21.10347023241006,
 19: 65.05004906771345,
 20: 6.717948717948718,
 21: 31.97588036661843,
 22: 69.336820083682,
 23: 5.536752422318743,
 24: 33143.0,
 25: 19.025832376578645,
 26: 32.58898721730581,
 27: 32.798614547253834,
 28: 2.869648036711546,
 29: 12.153648698203154,
 30: 66286.0,
 31: 2.710418711154727,
 32: 111.03182579564489,
 33: 136.6721649484536,
 34: 2285.7241379310344,
 35: 946.9428571428571,
 36: 13.24660271782574,
 37: 920.6388888888889,
 38: 212.4551282051282,
 39: 113.30940170940171,
 40: 90.9272976680384,
 41: 162.0684596577017,
 42: 4142.875,
 43: 50.2928679817

In [19]:
print(mlb.classes_.shape)
from pprint import pprint
pprint(class_weights.__len__())

(46,)
46


In [20]:
model = Sequential([
    Dense(100,activation='relu',input_shape=(X_train_tfidf.shape[1],)),
    Dense(100,activation='relu'),
    Dense(total_classes,activation='sigmoid')
])
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
X_train_tfidf_ordered = tf.sparse.reorder(reorder_sparse)

model.fit(X_train_tfidf_ordered, y_train, epochs=25, batch_size=128,class_weight=new_weights)

Epoch 1/25
 32/518 [>.............................] - ETA: 16:36 - loss: 6.7773 - accuracy: 0.0122

In [ ]:
model.evaluate(return_sparse(X_test_tfidf),y_test)

In [ ]:
model.summary()